# Stacking of Models

In [50]:
import numpy as np
import os
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GroupKFold
import lightgbm as lgb

# 1JHC

In [80]:
oof = pd.read_parquet('../type_results/1JHC/M037_0706_2029_1JHC_oof_0.5959MAE_-0.5176LMAE_catboost_2folds_500000iter_0.1lr.parquet')
oof = oof.drop('oof_preds', axis=1)
for o in [f for f in os.listdir('../type_results/1JHC') if 'oof' in f]:
    df = pd.read_parquet(f'../type_results/1JHC/{o}')
    model_num = o[:4]
    oof[model_num] = df['oof_preds']
    
sub = pd.read_parquet('../type_results/1JHC/M037_0706_2029_1JHC_submission_0.5959MAE_-0.5176LMAE_catboost_2folds_500000iter_0.1lr.parquet')
sub = oof.drop('scalar_coupling_constant', axis=1)
for o in [f for f in os.listdir('../type_results/1JHC') if 'oof' in f]:
    df = pd.read_parquet(f'../type_results/1JHC/{o}')
    model_num = o[:4]
    sub[model_num] = df['oof_preds']

In [82]:
train = pd.read_csv('../input/train.csv')

In [83]:
oof = pd.merge(train[['id','molecule_name']],
          oof,
         on=['id']
        )

In [84]:
X = oof[[m for m in oof.columns if 'M' in m]]
y = oof['scalar_coupling_constant']
G = oof['molecule_name']

In [89]:
folds = GroupKFold(n_splits=3)
oof_df = oof.copy()
for fold_n, (train_idx, valid_idx) in enumerate(folds.split(X, groups=G)):
    X_train = X.iloc[train_idx]
    X_valid = X.iloc[valid_idx]
    y_train = y.iloc[train_idx]
    y_valid = y.iloc[valid_idx]
    reg = lgb.LGBMRegressor(n_estimators=100000,
                            n_jobs=20,
                            learning_rate=0.001)
    reg.fit(X_train, y_train,
            eval_set=(X_valid, y_valid),
            verbose=1000,
           early_stopping_rounds=100,)
    oof_df.loc[oof_df.index.isin(valid_idx), 'stack_oof'] = reg.predict(X_valid)
#reg = lgb.LGBMRegressor(n_estimators=5000, n_jobs=20)
# reg.fit(X, y)
oof['stack_oof'] = reg.predict(X)

Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l2: 273.91
[200]	valid_0's l2: 224.718
[300]	valid_0's l2: 184.426
[400]	valid_0's l2: 151.413
[500]	valid_0's l2: 124.374
[600]	valid_0's l2: 102.225
[700]	valid_0's l2: 84.0796
[800]	valid_0's l2: 69.2147
[900]	valid_0's l2: 57.0381
[1000]	valid_0's l2: 47.0631
[1100]	valid_0's l2: 38.8906
[1200]	valid_0's l2: 32.1967
[1300]	valid_0's l2: 26.7134
[1400]	valid_0's l2: 22.2211
[1500]	valid_0's l2: 18.5401
[1600]	valid_0's l2: 15.5239
[1700]	valid_0's l2: 13.0526
[1800]	valid_0's l2: 11.0275
[1900]	valid_0's l2: 9.36783
[2000]	valid_0's l2: 8.00794
[2100]	valid_0's l2: 6.89359
[2200]	valid_0's l2: 5.98031
[2300]	valid_0's l2: 5.23192
[2400]	valid_0's l2: 4.61814
[2500]	valid_0's l2: 4.11561
[2600]	valid_0's l2: 3.70391
[2700]	valid_0's l2: 3.36546
[2800]	valid_0's l2: 3.08774
[2900]	valid_0's l2: 2.85978
[3000]	valid_0's l2: 2.67267
[3100]	valid_0's l2: 2.51901
[3200]	valid_0's l2: 2.39281
[3300]	valid_0's l2

In [90]:
mean_absolute_error(oof['scalar_coupling_constant'], oof['stack_oof'])

0.4760100496172856

In [87]:
mean_absolute_error(oof['scalar_coupling_constant'], oof['stack_oof'])

0.47552497176098835

In [88]:
mean_absolute_error(oof['scalar_coupling_constant'], oof['M047'])

0.4503699367015481

In [ ]:
reg.predict('')